In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install kaggle-environments

In [ ]:
%%writefile random_agent.py
import random
def random_agent(obs, config):
    return random.randint(0,2)

In [ ]:
%%writefile copy_agent.py
import random
def copy_opponent_agent(obs, config):
    if obs.step>0:
        return obs.lastOpponentAction
    else:
        random.randint(0,2)

In [ ]:
%%writefile random_forest_random.py
import random
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

actions =  np.empty((0,0), dtype = int)
observations =  np.empty((0,0), dtype = int)
total_reward = 0

def random_forest_random(observation, configuration):
    global actions, observations, total_reward
    
    if observation.step == 0:
        action = random.randint(0,2)
        actions = np.append(actions , [action])
        return action
    
    if observation.step == 1:
        action = random.randint(0,2)
        actions = np.append(actions , [action])
        observations = np.append(observations , [observation.lastOpponentAction])
        # Keep track of score
        winner = int((3 + actions[-1] - observation.lastOpponentAction) % 3);
        if winner == 1:
            total_reward = total_reward + 1
        elif winner == 2:
            total_reward = total_reward - 1        
        return action

    # Get Observation to make the tables (actions & obervations) even.
    observations = np.append(observations , [observation.lastOpponentAction])
    
    # Prepare Data for training
    # :-1 as we dont have feedback yet.
    X_train = np.vstack((actions[:-1], observations[:-1])).T
    
    # Create Y by rolling observations to bring future a step earlier 
    shifted_observations = np.roll(observations, -1)
    
    # trim rolled & last element from rolled observations
    y_train = shifted_observations[:-1].T
    
    # Set the history period. Long chains here will need a lot of time
    if len(X_train) > 25:
        random_window_size = 10 + random.randint(0,10)
        X_train = X_train[-random_window_size:]
        y_train = y_train[-random_window_size:]
   
    # Train a classifier model
    model = xgb1 = XGBClassifier(
 learning_rate =0.001,
 n_estimators=25,
 nthread=4)
    model.fit(X_train, y_train)

    # Predict
    X_test = np.empty((0,0), dtype = int)
    X_test = np.append(X_test, [int(actions[-1]), observation.lastOpponentAction])
    prediction = model.predict(X_test.reshape(1, -1))

    # Keep track of score
    winner = int((3 + actions[-1] - observation.lastOpponentAction) % 3);
    if winner == 1:
        total_reward = total_reward + 1
    elif winner == 2:
        total_reward = total_reward - 1
   
    # Prepare action
    action = int((prediction + 1) % 3)
    
    # If losing a bit then change strategy and break the patterns by playing a bit random
    if total_reward < -2:
        win_tie = random.randint(0,1)
        action = int((prediction + win_tie) % 3)

    # Update actions
    actions = np.append(actions , [action])

    # Action 
    return action 

In [ ]:
from kaggle_environments import make

In [ ]:
env = make("rps", configuration = {"episodeSteps": 1000})
env.run(["random_forest_random.py", "copy_agent.py"])

env.render(mode = 'ipython', width = 800, height = 800)